In [8]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import datetime

pd.set_option('display.max_columns', None)

In [9]:
data = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/Scorecard_Project/Internal_test_0_datasetForUnivariateVSExceptIV.csv', index_col=[0])

In [10]:
data

,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,2.657918,71948.0,0,-51.724408,35000.0,12.453608,16.02,0.49,3.0,561,-105.868637,1
1,-0.522740,12282.0,0,125.418365,1000.0,33.571609,11.14,0.08,2.0,504,947.746253,0
2,-0.522740,12438.0,3,77.828324,5500.0,-29.945313,12.87,0.44,3.0,635,-105.868637,1
3,-1.749761,79753.0,0,-51.724408,35000.0,-29.945313,15.23,0.44,2.0,675,-105.868637,1
4,2.657918,66135.0,1,-51.724408,35000.0,-29.945313,14.27,0.53,4.0,586,-105.868637,1
...,...,...,...,...,...,...,...,...,...,...,...,...
44995,1.090857,47971.0,6,-51.724408,15000.0,-29.945313,15.66,0.31,3.0,645,-105.868637,1
44996,1.090857,65800.0,17,-51.724408,9000.0,-22.266123,14.07,0.14,11.0,621,-105.868637,1
44997,1.090857,56942.0,7,-51.724408,2771.0,-41.855848,10.02,0.05,10.0,668,-105.868637,1
44998,-1.749761,33164.0,4,-51.724408,12000.0,33.571609,13.23,0.36,6.0,604,-105.868637,1


In [11]:
lstfeature = list(data.columns)

In [12]:
len(lstfeature)

12

In [13]:
lstfeature.remove('loan_status')

In [14]:
len(lstfeature)

11

In [15]:
features = []
pvalues = []
coeff = []

In [16]:
for i in lstfeature:
    features.append(i)
    modeldata = data[['loan_status', i]]
    Xtrain = modeldata[[i]]
    Xtrain = sm.add_constant(Xtrain)
    ytrain = modeldata[['loan_status']]

    try:
        log_reg = sm.Logit(ytrain, Xtrain).fit()
        pvalues.append(log_reg.pvalues[i])
        coeff.append(log_reg.params[i])
    except np.linalg.LinAlgError as err:
        if 'Singular matrix' in str(err):
            pvalues.append(1.0)
            coeff.append(0.0)
            continue
        else:
            raise

Optimization terminated successfully.
         Current function value: 0.529684
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.500417
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.529491
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.494694
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.524163
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.519494
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.472905
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.459470
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.529595
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.529677
  

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [17]:
dict = {'features': features, 'pvalues': pvalues}
df = pd.DataFrame(dict)

In [18]:
df

,features,pvalues
0,person_education,1.557062e-01
1,person_income,0.000000e+00
2,person_emp_exp,1.405091e-05
3,person_home_ownership,0.000000e+00
4,loan_amnt,2.637125e-113
5,loan_intent,3.860012e-195
6,loan_int_rate,0.000000e+00
7,loan_percent_income,0.000000e+00
8,cb_person_cred_hist_length,1.635328e-03
9,credit_score,1.047701e-01


In [19]:
df = df.sort_values(by = ['pvalues'], ascending = True).reset_index().drop(columns = ['index'])

In [20]:
df.to_csv(r'/content/drive/MyDrive/Colab Notebooks/Scorecard_Project/Internal_LR.csv')